#### Задача 1 (6 баллов). 

Напишем игру "ферма". Наш игрок - экземпляр класса "фермер", у которого есть имя, стартовая сумма денег и объект класса "ферма". У объекта класса "ферма" есть жилплощадь (для самого фермера), а также могут быть поле для картошки, например (с площадью), огород под огурцы, постройки для домашних животных (с вместимостью). Наш фермер может покупать семена и животных, собирать урожай и взаимодействовать с животными (коров можно доить, свиней можно зарезать и продать мясо). Игра консольная, поэтому придется выводить нужную информацию через запросы игрока (игрок может захотеть знать, сколько у него есть животных, какие у него есть огороды-поля и так далее). 

Можно не строго следовать описанию, а использовать фантазию. Если мне очень понравится, могу приплюсовать балл.

In [ ]:
# не успела :(

#### Задача 2 (6 баллов). 

Напишите программу - примитивного голосового помощника, задача которого - помогать пользователю изучать выбранный язык. Такая программа должна уметь следующие вещи: 

- разговаривать с пользователем голосом (и распознавать его речь)
- знать команды "переведи" и "зачитай вслух". Обе команды заставляют программу предложить пользователю ввести текст, который потом переводится и/или зачитывается программой. 
- знать команду "практика": программа предлагает пользователю небольшой текст, выбранный случайным образом из базы данных программы. Пользователь должен зачитать текст вслух, а программа должна сравнить распознанный голос пользователя с исходным текстом и сообщить, если не совпало. 
- опционально можно добавить функцию для ручного пополнения базы данных. 

Язык выбираете какой угодно (из поддерживаемых библиотеками, конечно). 

In [ ]:
import speech_recognition
import gtts
from playsound import playsound
import os
import re
import random
from googletrans import Translator, constants

In [ ]:
recognizer = speech_recognition.Recognizer()
microphone = speech_recognition.Microphone()
translator = Translator()

In [ ]:
def record_audio():
    with microphone:
        recognizer.adjust_for_ambient_noise(microphone, duration=2)        
        try:
            print("log: ждёт реплику")
            audio = recognizer.listen(microphone, 5, 5)
        except speech_recognition.WaitTimeoutError:
            print("log: микрофон не работает")
            return False
        return audio

def recognize_audio(audio, lang='ru'):
    if not audio:
        make_and_say_answer('Не вышло записать звук. Вы говорите тихо или с микрофоном неполадки')
        return False
    try:
        print("log: начал распознавание")
        recognized_data = recognizer.recognize_google(audio, language=lang).lower()
    except speech_recognition.UnknownValueError:
        make_and_say_answer('Я не поняла или не расслышала, повторите, пожалуйста')
        print(f'log: ошибка в распознавании, язык {lang}')
        return False
    except speech_recognition.RequestError:
        make_and_say_answer('Интернет не работает, а сам я ничего не умею')
        return False
    return recognized_data

In [ ]:
def make_and_say_answer(answer, lang='ru'):
    gtts.gTTS(answer, lang = lang, slow = False).save('bot_answer.mp3')
    playsound("bot_answer.mp3")
    os.remove('bot_answer.mp3') # и сразу удаляем, чтобы лишнее не плодить

In [ ]:
texts_to_practice = [
    'Ностальгия — это мечта о прошлом.',
    'Два банана стоят на берегу реки. Один говорит другому: "Давай поплывем по реке на другой берег!" Другой банан отвечает: "Нет, я боюсь, что мы замерзнем!"',
    'Два пингвина плавали в море, один говорит другому: "Ну что, пора домой?"',
    'Она пришла домой и увидела двух мужчин в кровати.',
    'Таксист приехал и попросил: "Денег или анекдот?"',
    'Шон барашек прыгал по лужам, пока не устал.',
    'Губка плавала и кричала: "Помогите!"',
    'Пряжа просила проводника: "Проведи меня до дома!"'
]
def practice():
    make_and_say_answer('Посмотрите в консоли текст, и когда будете готовы читать, напишите что угодно в поле для ввода', 'ru')
    text = random.choice(texts_to_practice)
    text_simple = ' '.join(re.sub(r'[^\w\s]', '', text.lower()).split())
    print('ВАШ ТЕКСТ: ' + text)
    input('ЧТО УГОДНО: ')
    recognized = recognize_audio(record_audio())
    if recognized != text_simple:
        print('Вы прочли: ' + recognized)
        print('А надо было: ' + text_simple)
        make_and_say_answer('Вы ошиблись, посмотрите в консоль', 'ru')
    else: 
        print('прочитано верно')
        make_and_say_answer('Вы такой молодец, всё прочитали!', 'ru')
        

In [ ]:
def translate(replica):
    recognized = recognize_audio(replica, 'ru')
    if recognized: 
        translation = translator.translate(recognized, dest = 'el').text
        print(f'реплика для перевода: {recognized}\nперевод: {str(translation)}')
        make_and_say_answer('Поняла! Переведу на греческий', 'ru')
        make_and_say_answer(translation, 'el')
    else: make_and_say_answer('У меня не получилось перевести. Попробуйте ещё раз', 'ru')

In [ ]:
next_step = True
make_and_say_answer('Привет, я - бот. Попросите меня повторить, перевести или попрактиковаться')
while next_step:
    # реплика пользователя
    recognized = recognize_audio(record_audio())
    if not recognized: continue
    print('реплика пользователя: ' + recognized)
    # переведи
    if re.match(r'перев[ео]д.*', recognized): 
        print('log: режим перевод')
        make_and_say_answer('Скажите фразу, а я - переведу')
        translate(record_audio())
    # переведи
    if re.match(r'.*читай.*', recognized): 
        print('log: режим чтение')
        make_and_say_answer('Напишите фразу в поле для ввода, а я - прочитаю')
        user_text = input('Введите текст: ')
        make_and_say_answer(user_text)
    # повтори
    elif re.match(r'повтор.*', recognized): 
        print('log: режим повтор')
        make_and_say_answer('Говорите, а я - повторю')
        recognized = recognize_audio(record_audio())
        if recognized: make_and_say_answer('Вы сказали:' + recognized)
    # практика
    elif re.match(r'.*(?:практик|трен[ие]р).*', recognized):
        # make_and_say_answer('Хорошо, внимание в консоль.')
        practice()
    # закончили 
    elif re.match(r'(?:[с]?топ|хватит|пока)', recognized):
        next_step = False
        make_and_say_answer('O нет!')
        print('log: завершил работу')
    else:
        make_and_say_answer('Чего вы хотите? Вы сказали:' + recognized)

#### Задача 3 (1 балл). 

Возьмите любую лингвистическую статью по синтаксису (можно найти на scholar.google.com) и сверстайте в латехе любые три синтаксических дерева оттуда. 

In [ ]:
# из этой книжки
# http://ndl.ethernet.edu.et/bitstream/123456789/88321/1/Andrew_Radford%5D_English_Syntax_An_Introduction%28BookFi.org%29%20%281%29.pdf

In [ ]:
# результат в latex_res.pdf
# а вот как я рисую:

\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage[russian]{babel}
\usepackage[utf8]{inputenc}

\usepackage{soul}
\usepackage{tikz-qtree}
\tikzstyle{bag} = [align=center]

\begin{document}
% http://ndl.ethernet.edu.et/bitstream/123456789/88321/1/Andrew_Radford%5D_English_Syntax_An_Introduction%28BookFi.org%29%20%281%29.pdf
1. стр. 190, пример (86) \bigbreak

\begin{tikzpicture}
    \node (is-root) {VP'}
    [sibling distance=3cm]
    child {
        node[bag] {T \\ HAVE \\ {[Pres-Tns]} \\ {[\textit{u-Pers}]}\\ {[\textit{u-Num}]}}
    }
    child {
        node {vP}
        child {
            node[bag] {PRN \\ YOU \\ {[2-Pers]} \\ {[PL-Num]} \\ {[\textit{u-Case}]}}
        }
        child {
            node {v'}
            child {
                node {v}
                child {
                    node[bag] {ø+upset \\ {[\st{\textit{3-Pers}}]} \\ {[\st{\textit{Pl-Num}}]}}
                }
            }
            child {
                node {VP}
                child {
                    node[bag] {V \\ \st{upset}}
                }
                child {
                    node[bag] {PRN \\ them \\ {[3-Pers]} \\ {[PL-Num]} \\ {[\st{\textit{Acc-Case}}]}}
                }
            }
        }
    }
\end{tikzpicture} \bigbreak
2. стр. 194, пример (3) \bigbreak
\begin{tikzpicture}
 \node (is-root) {vP}
    [sibling distance=3cm]
    child {
        node[bag] {DP \\ ø Ruritania}        
    }
    child {
        node {v'}
        child {
            node[bag] {v \\ ø+withdraw}
        }
        child {
            node {VP}
            child{
                node[bag] {QP \\ ø troops}
            }
            child {
                node {V'}
                child {
                    node[bag] {V \\ \textit{withdraw}}
                }
                child {
                    node[bag] {PP \\ from ø Utopia}
                }
            }
        }
    }
\end{tikzpicture} \bigbreak
3. стр. 195, пример (5) \bigbreak
\begin{tikzpicture}
 \node (is-root) {TP}
    [sibling distance=4cm]
    child {
        node[bag] {DP \\ ø Ruritania}        
    }
    child {
        node {T}
            child {
                 node[bag] {T \\ will}
            }
            child {
                node {vP}
                child {
                    node[bag] {DP \\ ø Ruritania}
                }
                child {
                    node {v'}
                    child {
                        node[bag] {v \\ ø+withdraw}
                    }
                    child {
                        node[bag] {PP \\ \st{withdraw} ø troops from ø Utopia}
                    }
                }
            }
    }
\end{tikzpicture}
\end{document}

#### Задача 4 (1 балл). 

Напишите функцию, которая строит график для n самых частотных слов в тексте. Сделайте интерактивный график, где можно выбирать количество n слов.

In [358]:
text = '''Зацепилась тут в комментах за:

«Если партнер себе нашёл какое-то хобби, которое начало его затягивать, и явно не развивает его, то очевидно, что ему вправить вовремя мозги»

Вот вопрос, что такое «явно» и что такое «очевидно». И что такое «вправить мозги».
Если попробовать посмотреть внимательно, многие явные вещи - не то, чем кажутся, или вообще не существуют. В смысле «явно» - не аргумент.
Приведу пример, как «близкий делает явное не то, и, очевидно, пора ему вправить мозги».

Когда-то папе показалось, что я недостаточно рьяно готовлюсь к школьным выпускным экзаменам, но при этом играю на гитаре. И он сказал «я сейчас сломаю и выброшу эту гитару». Вообще, он золотой и совершенно беззлобный человек. И училась я хорошо. И гитару эту именно он мне купил - в эпоху тотального отсутствия всего. Это были страшные корявые и твердые дрова, но их появление (в седьмом классе) было поворотом в моей судьбе. Это была _моя_ гитара - в родительском доме, купленная родителями, но моя. Ничего он, конечно, не сделал, было сказано не подумав и в сердцах, но запомнилось. Вот видите, сколько лет прошло, а я помню. Потому, что это была угроза недопустимого насилия над моей жизнью. И во мне было внутреннее ощущение: «он не имеет права!». Экзаменационные знания (например, идеологизированное и лживое обществоведение) мне не сказать, что пригодились, хотя вроде образование. Но сдала я их, что ж не сдать.
Не мог папа предвидеть, что эта дровяная корявая гитара - на самом деле сейчас главный предмет в моей жизни и пропуск в предназначенное мне будущее, а не какая-то вредная фигня, отвлекающая от главного.
У меня, как вы знаете, нет начального музыкального образования. Я самоучка, на той самой страшной гитаре открывшая и придумавшая музыку самостоятельно и без учебников. Родители не понимали и не предвидели моего будущего, мне пришлось самой наверстывать отсутствие музыкальной школы. Дальше у меня появилось среднее и высшее музыкальное, но так и не появилось начального. Как кто-то сказал, у тебя сейчас может быть бентли и самолет, но в детстве всё равно не было велосипеда.
Так к чему я. Если близкий пляшет в простынях и потом едет валяться в грязи на дурацком фестивале буто, если он бежит спасать собачек или едет на тренинг по, фигурально выражаясь, арке трансформаций - совершенно не факт, что его надо запереть и сказать: «ты останешься, потому что я так сказал». И нельзя сказать: «я выкину сейчас твои арки и костюмы, иди вари борщи».
Для меня это означает «ты моя вещь, и должна делать что я скажу». И именно от такого у меня подгорает и вскипает протест.
Во-первых, даже действуя из лучших побуждений, доброжелатель не знает и не может предвидеть, что вам на самом деле глубинно нужно. Возможно, нужно нечто странное, но это ваша жизнь, а не его.
Второе. Люди, вторгающиеся с благими намерениями, всегда действуют из своего близорукого интереса. Из своих выгоды, удобства, спокойствия, амбиций, планов, которые в данный момент уперлись в вас. Хотя в основном не осознают и не признают этого. Просто мы, люди, так вообще устроены, и ничего тут не попишешь.
И да, близким может быть удобнее, чтобы вы сидели дома и варили борщи (читали книжки, готовились к экзамену), а не лезли на Эльбрус. Спокойнее, если вы сбагрены в брак и всё как у людей (к левому человеку, с которым вы несчастны). Амбициознее, если вы получаете престижное образование, пусть вам неинтересное и неполезное. Бывает, вы, напротив, хотите читать книжки и варить борщи, а вас волокут на соревнования, конкурсы, экзамены и кастинги.
Причинить добро это называется.
Никто на самом деле не может лучше вас знать, что вам нужно. Но всем удобнее, чтобы вы действовали как им надо, были понятны, предсказуемы и в рамках. Все вас мерят своей меркой, и их можно понять и простить. Но планы близких на вас - только дополнительный фактор. Главное - чего вы хотите сами. Если вы взрослый проявленный и осознанный человек, вы не будете ломать дров. Но вполне можете их наломать, если вам слишком долго не дают быть собой. Тогда да, здравствуй самоповреждение, разрушительное поведение, зависимости и депрессии.
Кстати, для меня было новым ощущением, когда мой бойфренд, наконец-то не нарцисс и не абьюзер, на тот или иной вопрос «как мне лучше поступить?» отвечал: «посиди, прислушайся, как ты почувствуешь». Это не была отмазка, он готов был сопроводить мои процессы, обсудить их, побыть рядом. Но выбор оставлял за мной.
Как-то многие десятилетия до того никто не прислушивался к тонким нюансам моих желаний. Всегда меня водили строем, забивали как гвоздь, вписывали в рамки, убеждали быть как все, или как надо конкретному человеку. Или уже у меня накапливалась атомная энергия, и я все нафиг взрывала. Но, оказывается, можно обойтись без резких переломов жизни, если прислушиваться на более раннем этапе к своим чуть более тонким ощущениям. И позволять это делать другим.
Этот разговор не так прост. Есть действительно опасные вещи, от которых близкого стоит пытаться отговорить. Как минимум, это всё, что связано с зависимостями, с самоповреждением. И да, со связями с опасными людьми. В том числе, с попаданием в отношения с абьюзером. Один из маркеров потенциального абьюзера - ощущение партнера своей вещью и собственностью с правом распоряжаться его жизненным выбором.
Но мы не всегда можем помочь.
Возможность вмешательства в чужую жизнь, даже если это жизнь близкого, у нас очень ограничена.
В частности, даже в случае очевидной связи близкого с абьюзером, как правило, окружающие не могут помочь. Могут только аккуратно высказать свое мнение и оказать поддержку, если к ним обратятся. Сам решит уйти - помочь.
Но в целом другой человек - это вселенная, над которой никак мы не властны. Мы можем только наблюдать, в том числе ошибки и опасные глупости. Может, людям позарез надо пройти уроки именно в таком виде? Может надо от судьбы получить именно это? Реализовать дурацкие желания? Пойти по непонятной тропе, которая, может привести неведомо куда? Но это их тропа. А мы их зачем-то будем утаскивать с такого интересного аттракциона. Это только накапливает ярость, нереализованность, раздражение, или приводит к апатии.
Получается, с близкими мы можем только обсуждать и договариваться, разъяснять и доказывать свою точку зрения.
Но никаких «потому, что я так сказал». Вот об этом я хотела поговорить, задавая вопрос, что такое «явно», «не то» и как вообще можно «вправить мозги».
'''

In [ ]:
from nltk.probability import FreqDist
import nltk
import matplotlib.pyplot as plt
import re

text_tokens = nltk.word_tokenize(re.sub(r'[^\w\s]', '', text.lower()))
fdist = FreqDist(nltk.Text(text_tokens))

In [ ]:
def draw_often_words(count):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    X = [elem[0] for elem in fdist.most_common(count)]
    Y = [elem[1] for elem in fdist.most_common(count)]
    ax.plot(X, Y, '-ro')
    plt.show()
draw_often_words(6)

#### Задача 5 (1 балл).

Напишите программу, которая будет запрашивать у пользователя, какой из трех любых (выберите сами) признаков WALS он бы хотел посмотреть, и отрисовывать карту с WALS с этим признаком при помощи folium. Маркеры разных значений признака должны иметь разный цвет и иконку. 

In [ ]:
import json
import folium
from folium.features import GeoJson

feature_index = ''
while not feature_index in set(range(3)):
    try:
        feature_index = int(input('Введите\n\t0 — чтобы посмотреть на карту про тоны\n\t1 — чтобы посмотреть карту про неопределённые местоимения\n\t2 — чтобы посмотреть карту про род\n'))
    except:
        pass

var_maps = [
    ['maps/13A_Complex tone system.geojson', 'maps/13A_No tones.geojson', 'maps/13A_Simple tone system.geojson'],
    ['maps/46A_Generic-noun-based.geojson', 'maps/46A_Interrogative-based.geojson', 'maps/46A_Special.geojson'],
    ['maps/32A_No gender.geojson', 'maps/32A_Semantic and formal.geojson', 'maps/32A_Semantic.geojson']
]
world_map = folium.Map(location=[50, 90], zoom_start=2)
colours = ["green", 'orange', 'black']
spec_vars = var_maps[feature_index]
for i in range(3):
    folium.GeoJson(
        json.load(open(spec_vars[i])),
        marker=folium.Marker([50, 90], icon=folium.Icon(color=colours[i],icon="circle", prefix='fa')),
    ).add_to(world_map)
world_map

#### 6. Задачи по pandas - в сумме 10 баллов

В папке Data находится информация о студентах. Всего 10 групп студентов. Файлы делятся на две категории:
    * Students_info_i - информация о студентах из группы i
    * Students_marks_i - оценки студентов из группы i за экзамены

In [ ]:
import pandas as pd
import os
from zipfile import ZipFile

Задание 1. Соберите всю информацию о студентах в одну таблицу df. В получившейся таблице должна быть информация и оценки всех студентов из всех групп. Напечатайте несколько строк таблицы для демонстрации результата.¶

In [ ]:
# распаковать архив

with ZipFile('Data.zip', 'r') as zip_file:
    zip_file.extractall('.')

In [ ]:
part_dfs_students = [pd.read_csv('./Data/' + file) for file in os.listdir('./Data') if file.startswith('Students_info_')]
df_students = pd.concat(part_dfs_students) # df про студентов
# df_students.sample(10)

In [ ]:
part_dfs_marks = [pd.read_csv('./Data/' + file) for file in os.listdir('./Data') if file.startswith('Students_marks_')]
df_marks = pd.concat(part_dfs_marks) # df про оценки
# df_marks.sample(10)

In [ ]:
df = pd.merge(df_students, df_marks) # всё, что мы знаем о студентах
df.sample(10)

Задание 2. Удалите столбец index у полученной таблицы. Напечатайте первые 10 строк таблицы.

In [ ]:
df = df.drop(columns = ['index'])
df.head(10)

Задание 3. Выведите на экран размеры полученной таблицы

In [ ]:
print(f'Размеры таблицы — {df.shape}, где {df.shape[0]} — количество строк, а {df.shape[1]} — количество столбцов.')

Задание 4. Выведите на экран статистические характеристики числовых столбцов таблицы (минимум, максимум, среднее значение, стандартное отклонение)

In [ ]:
def form_stat_str(column_title, df):
    return f'''Для {column_title}:
- минимум: {df[column_title].min()}
- максимум: {df[column_title].max()}
- среднее значение: {df[column_title].mean()}
- стандартное отклонение: {df[column_title].std()}'''

print(form_stat_str('math score', df)
     + '\n\n' + form_stat_str('reading score', df)
     + '\n\n' + form_stat_str('writing score', df))

Задание 5. Проверьте, есть ли в таблице пропущенные значения

In [ ]:
df.isnull().sum()
# Нет, какая красота

Задание 6. Выведите на экран средние баллы студентов по каждому предмету (math, reading, writing)

In [ ]:
# кажется, я что-то недопоняла, потому что это уже было в задании 4
print(f'''Средний балл...
- по математике (math): {df['math score'].mean()} 
- по чтению (reading): {df['reading score'].mean()}
- по письму (writing): {df['writing score'].mean()}''')

Задание 7. Как зависят оценки от того, проходил ли студент курс для подготовки к сдаче экзамена (test preparation course)? Выведите на экран для каждого предмета в отдельности средний балл студентов, проходивших курс для подготовки к экзамену и не проходивших курс.

In [ ]:
df[df['test preparation course'] == 'completed']['math score'].mean()

print(f'''Средний балл...
- по математике (math):\t{df[df['test preparation course'] == 'completed']['math score'].mean()} у прошедших курс\tи {df[df['test preparation course'] == 'none']['math score'].mean()} у не прошедших
- по чтению (reading):\t{df[df['test preparation course'] == 'completed']['reading score'].mean()} у прошедших курс\tи {df[df['test preparation course'] == 'none']['reading score'].mean()} у не прошедших
- по письму (writing):\t{df[df['test preparation course'] == 'completed']['writing score'].mean()} у прошедших курс\tи {df[df['test preparation course'] == 'none']['writing score'].mean()} у не прошедших''')

# получается, проходившие курс подготовки в среднем лучше сдавали экзамен, причём все три предмема

Задание 8. Выведите на экран все различные значения из столбца lunch.

In [ ]:
# можно так:
df['lunch'].unique()

In [ ]:
# чтобы были только все значения проще всего сделать
set(df['lunch'])

Задание 9. Переименуйте колонку "parental level of education" в "education", а "test preparation course" в "test preparation" с помощью метода pandas rename.

In [ ]:
df = df.rename(columns = {'parental level of education': 'education'})

**Зафиксируем минимальный балл для сдачи экзамена**

In [ ]:
passmark = 50

Задание 10. Ответьте на вопросы:

    * Какая доля студентов сдала экзамен по математике (passmark > 50)?
    * Какая доля студентов, проходивших курс подготовки к экзамену, сдала экзамен по математике?
    * Какая доля женщин, не проходивших курс подготовки к экзамену, не сдала экзамен по математике? 

In [ ]:
just_percent = df[df['math score'] > passmark].count()[0] / len(df['math score']) * 100
# можно было бы использовать count(), но я, честно говоря, не понимаю зачем
# just_percent = df[df['math score'] > passmark].count()[0] / len(df['math score']) * 100 

after_course_percent = round(len(df[(df['math score'] > passmark) & (df['test preparation course'] == 'completed')]) / len(df[df['test preparation course'] == 'completed']) * 100, 2)
fail_after_course_percent = round(len(df[(df['math score'] > passmark) & (df['test preparation course'] == 'none')]) / len(df[df['test preparation course'] == 'none']) * 100, 2)

print(f"""Из всех студентов {df[df['math score'] > passmark].count()[0] / len(df['math score']) * 100}% сдали экзамен по математике,

причём среди проходивших курс подготовки к экзамену сдали {after_course_percent}%,

а среди не проходивших — {fail_after_course_percent}%""")

Задание 11. С помощью groupby выполните задания ниже. Также выведите время выполнения каждого из заданий.

    * Для каждой этнической группы выведите средний балл за экзамен по чтению
    * Для каждого уровня образования выведите минимальный балл за экзамен по письму

In [ ]:
# Для каждой этнической группы выведите средний балл за экзамен по чтению
print(df.groupby('race/ethnicity')['reading score'].mean())

In [ ]:
# Для каждого уровня образования выведите минимальный балл за экзамен по письму
print(df.groupby('education')['writing score'].min())


Задание 12. Выполните задание 11 с помощью циклов. Сравните время выполнения.

In [ ]:
# Для каждой этнической группы выведите средний балл за экзамен по чтению
def mean_for_reading_in_races(df):
    races = dict()
    for race in set(df['race/ethnicity']):
        races[race] = []
    for index, row in df.iterrows():
        races[row['race/ethnicity']].append(row['reading score'])
    for race in races:
        races[race] = sum(races[race]) / len(races[race])
    return races
# print(mean_for_reading_in_races(df))

%timeit (mean_for_reading_in_races(df))

In [ ]:
# считаем то же с groupby
%timeit df.groupby('race/ethnicity')['reading score'].mean()

In [ ]:
# Для каждого уровня образования выведите минимальный балл за экзамен по письму
def mean_for_writing_in_educ_levels(df):
    educ_levels = dict()
    for educ_level in set(df['education']):
        educ_levels[educ_level] = 100
    for index, row in df.iterrows():
        if row['writing score'] < educ_levels[row['education']]:
            educ_levels[row['education']] = row['writing score']
    return educ_levels

# print(mean_for_writing_in_educ_levels(df))
%timeit (mean_for_writing_in_educ_levels(df))

In [ ]:
# считаем то же с groupby
%timeit df.groupby('education')['writing score'].min()

In [ ]:
# Ну что, разница большая, конечно.

# Наверное можно как-то более оптимально сделать циклами, но так хорошо точно не будет

Задание 13. Выведите на экран средние баллы студентов по каждому предмету в зависимости от пола и уровня образования. То есть должно получиться количество групп, равных 2 * (число уровней образования), и для каждой такой группы выыведите средний балл по каждому из предметов.

Это можно сделать с помощью сводных таблиц (pivot_table).

In [ ]:
dropped_df =  df.drop(columns = ['race/ethnicity', 'lunch', 'group', 'test preparation course']) # убираем нечисловые колонки из df
pd.pivot_table(dropped_df, index=['gender', 'education'])

Задание 14. Сколько студентов успешно сдали экзамен по математике?

Создайте новый столбец в таблице df под названием Math_PassStatus и запишите в него F, если студент не сдал экзамен по математике (балл за экзамен < passmark), и P иначе.

Посчитайте количество студентов, сдавших и не сдавших экзамен по математике.

Сделайте аналогичные шаги для экзаменов по чтению и письму.

In [ ]:
df['Math_PassStatus'] = df['math score'].apply(lambda mark: 'F' if mark < passmark else 'P') # сделали и заполнили столбец
print(f"Математику не сдали {len(df[df['Math_PassStatus'] == 'F'])} студентов (из {len(df)})")

In [ ]:
df['Reading_PassStatus'] = df['reading score'].apply(lambda mark: 'F' if mark < passmark else 'P') # сделали и заполнили столбец
print(f"Чтение не сдали {len(df[df['Reading_PassStatus'] == 'F'])} студентов (из {len(df)})")

In [ ]:
df['Writing_PassStatus'] = df['writing score'].apply(lambda mark: 'F' if mark < passmark else 'P') # сделали и заполнили столбец
print(f"Письмо не сдали {len(df[df['Writing_PassStatus'] == 'F'])} студентов (из {len(df)})")

Задание 15. Сколько студентов успешно сдали все экзамены?

Создайте столбец OverAll_PassStatus и запишите в него для каждого студента 'F', если студент не сдал хотя бы один из трех экзаменов, а иначе 'P'.

Посчитайте количество студентов, которые сдали все экзамены.

In [ ]:
df['OverAll_PassStatus'] = df.apply(lambda row: 'F' if (row['math score'] < passmark or row['reading score'] < passmark or row['writing score'] < passmark) else 'P', axis=1) # сделали и заполнили столбец
print(f"Все экзамены сдали {len(df[df['OverAll_PassStatus'] == 'P'])} студентов (из {len(df)})")

Задание 16. Переведем баллы в оценки

Система перевода баллов в оценки

      [90;100] = A
      [80;90) = B
      [70;80) = C
      [60;70) = D
      [50;60) = E
      [0;50) = F (Fail)

Создайте вспомогательную функцию, которая будет по среднему баллу за три экзамена выставлять оценку студенту по данным выше критериям.

Создайте столбец Grade и запишите в него оценку каждого студента.

Выведите количество студентов, получивших каждую из оценок.

In [ ]:
def GetGrade(average_mark):
    marks_letters = {90: 'A', 80: 'B', 70: 'C', 60: 'D', 50: 'E'}
    for letter_min_value in marks_letters:
        if average_mark >= letter_min_value: return marks_letters[letter_min_value]
    return 'F'

df['Gemeral Grade'] = df.apply(lambda row: GetGrade((row['math score'] + row['reading score'] + row['writing score']) / 3), axis=1)
# df

#### Задача 7 (5 баллов). 

Напишите примитивный краулер, который с помощью библиотек requests & beautifulsoup4 будет собирать тексты с сайта proza.ru. 

In [ ]:
import re
import requests
from bs4 import BeautifulSoup as bs

In [ ]:
try:
    os.makedirs('proza_texts')
except OSError as e:
    if e.errno == 17: print('Эта папка уже существует')
    else: raise

In [ ]:
def check_link(page, viewed, dir_title, text_counter):
    viewed.add(page)
    soup = bs(requests.get(page).text, 'html.parser')
    
    link_ends_date = re.match(r'https://proza.ru/\d{4}/(\d\d/\d\d/.*)', page)
    if re.match(r'https://proza.ru/\d{4}/(\d\d/\d\d/.*)', page):
        with open(dir_title + '/' + link_ends_date[1].replace('/', '-') + '.txt', 'w') as f:
            f.write('\n'.join((elem.text for elem in soup.find_all("div", {"class": "text"}))))
    links = []
    for link in soup.find_all('a', attrs={'href': re.compile(r'^/.+')}):
        l = 'https://proza.ru' + link.get('href')
        if l not in viewed:
            links.append(l)
    if links:
        for link in links:
            if sum(1 if file.endswith('.txt') else 0 for file in os.listdir(dir_title)) >= text_counter: return
            if re.search('board|login|help|topic|type|about|addrec|complain|readers', link): continue
            sleep(0.5)
            check_link(link, viewed, dir_title, text_counter)
    else:
        return 

In [ ]:
viewedlinks = set()
check_link('https://proza.ru/avtor/karinedkar', viewedlinks, 'proza_texts', 5)
# os.listdir('proza_texts')

#### Задача 8 (5 баллов). 

Некоторое лингвистическое исследование показало, что в дневниковых записях и подобных текстах (н-р, блоги ЖЖ) на 100 личных местоимений местоимения распределяются следующим образом:

    я - 50 раз
    ты - 5 раз
    он/она/оно - 25 раз
    мы - 3 раза
    вы - 2 раза
    они - 15 раз
    
Возьмите любой достаточно большой набор текстов (блоги ЖЖ, дневниковые записи) и проверьте, действительно ли в нем такое распределение местоимений. 

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import re
import json
import time

In [ ]:
# ссылки на конкретные посты
def get_posts_links(page_from):
    links = set()
    soup = bs(requests.get(page_from).text, 'html.parser')
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            match = re.match(r'https:\/\/ylgur\.livejournal\.com\/\d*\.html', href)
            if match and match[0] == href:
                links.add(href)
    return links
# get_posts_links('https://ylgur.livejournal.com/') # например для первой страницы

In [ ]:
def get_text_of_post(link):
    soup = bs(requests.get(link).text, 'html.parser')
    return soup.find('div', class_='asset-body').get_text()[6:] # [6:] потому что я не знаю, как вынуть (и выкинуть) оттуда "i-ljuser-username"

In [ ]:
posts = set()
for i in range(200, 400, 10): # можно вызять и больше, гдавное, чтобы число делилось на 10 (ну и чтобы в блоге вообще столько было)
    posts = posts | get_posts_links(f'https://ylgur.livejournal.com/?skip={i}')
    time.sleep(0.2)
# posts

In [ ]:
# собираем корпус
with open('corpora.txt', 'w', encoding='utf-8') as f:
    f.write(' '.join(map(get_text_of_post, posts)))

In [ ]:
# лемматизация
!mystem -n -c -i -d --format json corpora.txt lem_res.json

In [ ]:
# словарик с леммами
with open('lem_res.json', 'r', encoding='utf-8') as f:
    lem_dict = [json.loads(line) for line in f.readlines() if line != '{"text":" "}\n']

In [ ]:
prons = {'я': 0, 'ты': 0, 'он(-а/-о)': 0, 'мы': 0, 'вы': 0, 'они': 0}

# считаем
for elem in lem_dict:
    if 'analysis' in elem and len(elem['analysis']) and elem['analysis'][0]['lex'] in prons:
        prons[elem['analysis'][0]['lex']] += 1
    elif 'analysis' in elem and len(elem['analysis']) and elem['analysis'][0]['lex'] in ['он', 'она', 'оно']:
        prons['он(-а/-о)'] += 1
# prons

In [ ]:
all_count = sum(prons.values())

# выводим процент
for key in prons:
    if key == 'он(-а/-о)':
        print(f'"{key}":\t примерно {round(prons[key] / all_count * 100)} на 100 личных местоимений') # мне лень было заморачиваться
    else:
        print(f'"{key}":\t\t примерно {round(prons[key] / all_count * 100)} на 100 личных местоимений') 
# Итог: ну как будто в этом блоге больше "нас" и "вас", чем должно быть — что ж, бывает 